In [1]:
import numpy as np
import pandas as pd
import os
from glob import glob
import matplotlib.pyplot as plt

from thefuzz import fuzz, process

In [2]:
ipg_ids = pd.read_excel('../raw/ipg_wos_ids.ods', engine='odf')
wos_mask = ipg_ids.iloc[:,4:].notna().values
wos = ipg_ids.iloc[:,4:].values[wos_mask]
ipg_names = ipg_ids['IPG'].values
print(wos.shape)

(176,)


In [15]:
print('AI = ({})'.format(' OR '.join(wos)) )

AI = (CBE-4324-2022 OR CBN-5956-2022 OR H-8368-2015 OR KFM-0458-2024 OR G-1969-2015 OR KKR-9775-2024 OR KQR-2448-2024 OR MFU-1899-2025 OR ELB-9477-2022 OR IJL-6492-2023 OR FZV-5395-2022 OR EPN-2939-2022 OR KAA-0528-2024 OR EOI-9585-2022 OR IAX-5680-2023 OR CHP-5632-2022 OR KCM-1031-2024 OR JBW-9506-2023 OR KCT-4444-2024 OR HGT-9595-2022 OR ENO-9327-2022 OR CHO-4713-2022 OR GBC-6210-2022 OR N-8209-2013 OR DSS-1130-2022 OR IRT-7105-2023 OR DSJ-9947-2022 OR CLX-7732-2022 OR IBO-8695-2023 OR CNX-6601-2022 OR ICJ-0558-2023 OR COH-7797-2022 OR DVG-9458-2022 OR FZB-8921-2022 OR CQC-0997-2022 OR CBY-5175-2022 OR MFB-1084-2025 OR IKP-2914-2023 OR HVL-8243-2023 OR FZL-2254-2022 OR CQW-2740-2022 OR KOX-9064-2024 OR CPZ-2723-2022 OR IRM-7363-2023 OR ICU-1372-2023 OR EXP-0201-2022 OR AFM-9644-2022 OR IAI-2779-2023 OR GCA-3334-2022 OR CQU-8893-2022 OR CSS-5020-2022 OR HIB-8150-2022 OR CSI-5103-2022 OR CSM-0261-2022 OR CUG-6635-2022 OR EYP-1172-2022 OR KKL-9484-2024 OR FYK-2959-2022 OR ILP-5061-2023 

In [13]:
institutions = ['Lincoln University - Missouri',
                'University of Missouri Columbia',
                'University of Missouri System',
                'United States Department of Agriculture (USDA)',
                'Donald Danforth Plant Science Center']
institutions = ' OR '.join(institutions)
og = 'OG = (' + institutions +')'
og

'OG = (Lincoln University - Missouri OR University of Missouri Columbia OR University of Missouri System OR United States Department of Agriculture (USDA) OR Donald Danforth Plant Science Center)'

In [16]:
for i in range(len(ipg_ids)):
    print(i, end='\t')
    print(ipg_ids.iloc[i, 1], end=':\t')
    print('AI = ({}) AND {}'.format(' OR '.join(ipg_ids.iloc[i, [False,False,False,False]+wos_mask[i].tolist()]), og ) )

0	Emily R. Althoff:	AI = (CBE-4324-2022) AND OG = (Lincoln University - Missouri OR University of Missouri Columbia OR University of Missouri System OR United States Department of Agriculture (USDA) OR Donald Danforth Plant Science Center)
1	Ruthie Angelovici:	AI = (CBN-5956-2022) AND OG = (Lincoln University - Missouri OR University of Missouri Columbia OR University of Missouri System OR United States Department of Agriculture (USDA) OR Donald Danforth Plant Science Center)
2	Jaime Barros-Rios:	AI = (H-8368-2015 OR KFM-0458-2024) AND OG = (Lincoln University - Missouri OR University of Missouri Columbia OR University of Missouri System OR United States Department of Agriculture (USDA) OR Donald Danforth Plant Science Center)
3	Norman B. Best:	AI = (G-1969-2015 OR KKR-9775-2024 OR KQR-2448-2024 OR MFU-1899-2025) AND OG = (Lincoln University - Missouri OR University of Missouri Columbia OR University of Missouri System OR United States Department of Agriculture (USDA) OR Donald Danfort

In [49]:
numfiles = len(glob('../raw/savedrecs*.txt'))
txtdata = [None for _ in range(numfiles)]
for i in range(len(txtdata)):
    txtdata[i] = pd.read_table('../raw/savedrecs-{}.txt'.format(i))

data = pd.concat(txtdata[-(numfiles-len(ipg_ids)):], ignore_index=True)
print(data.shape)

(4194, 71)


In [80]:
mergecols = data.columns[np.sum(data.notna().values, axis=0) == len(data)].tolist()

In [96]:
i,j = 56,55
merge = pd.merge(txtdata[i], txtdata[j], how='inner', on=mergecols)
print(merge.shape)
print(merge.index)

(2, 119)
RangeIndex(start=0, stop=2, step=1)


In [97]:
k = 0
for v in range(len(mergecols)):
    print(mergecols[v], merge.loc[k, mergecols[v]], sep='\t')

PT	J
AU	Brechenmacher, L; Lei, ZT; Libault, M; Findley, S; Sugawara, M; Sadowsky, MJ; Sumner, LW; Stacey, G
AF	Brechenmacher, Laurent; Lei, Zhentian; Libault, Marc; Findley, Seth; Sugawara, Masayuki; Sadowsky, Michael J.; Sumner, Lloyd W.; Stacey, Gary
TI	Soybean Metabolites Regulated in Root Hairs in Response to the Symbiotic Bacterium <i>Bradyrhizobium japonicum</i>
SO	PLANT PHYSIOLOGY
LA	English
DT	Article
NR	154
TC	109
Z9	133
U1	3
U2	65
PU	OXFORD UNIV PRESS INC
PI	CARY
PA	JOURNALS DEPT, 2001 EVANS RD, CARY, NC 27513 USA
PY	2010
PG	15
WC	Plant Sciences
WE	Science Citation Index Expanded (SCI-EXPANDED)
SC	Plant Sciences
GA	634FQ
DA	2024-08-31
UT	WOS:000280566000030


In [27]:
i = 5
for name in data.loc[i, 'AF'].split('; '):
    print(name, process.extractOne(name, ipg_names, scorer=fuzz.partial_token_sort_ratio), sep='\t')

WANG, YF	('Bing Yang', 62)
RICHARDSON, SG	('Richard A. Ferrieri', 70)
SHARP, RE	('Robert E. Sharp', 88)
